In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sts

Загружаем выборки.

In [2]:
data1 = pd.read_excel('Data_Extract_From_Gender_Statistics.xlsx', encoding = 'utf8')

data2 = pd.read_excel('Data_Extract_From_Health_Nutrition_and_Population_Statistics.xlsx', encoding='utf8')

data3 = pd.read_excel('Data_Extract_From_Millennium_Development_Goals.xlsx', encoding='utf8')

data_cnt = pd.read_csv('all.csv', encoding='utf8')

Очищаем данные от больших пропусков, точек и пр.

In [3]:
class Analyst:
       def __init__(self, data, width=100, height=100):
            self.data = data

In [4]:
def Fillnan(data, years_c):
    for name in years_c:
        data[name] = data[name].apply(lambda x: np.nan if x==('..') else float(x))
    return data
    
def Dropempt(data, years_c):
    tmp=[]
    for ind, item in enumerate(data[years_c].as_matrix()):
        if np.nansum(item) == 0:
            tmp.append(ind)
    data = data.drop(data.index[tmp]).reset_index()
    return data.drop('index', axis=1)
    
def Clearing(data):
    years_c = [item for ind, item in enumerate(np.array(data.columns)) if ind not in (range(0,4))]
    if data['Country Code'].get_value(len(data)-5) == np.nan:
        data=data.drop(data.index[[range(len(data)-5, len(data))]]).reset_index()
    data = Dropempt(Fillnan(data, years_c), years_c)
    return data

Сортировка по регионам.

In [5]:
def Make_region(code, reg='region'):
    #print code
    if code in data_cnt['name'].values:
        #print code
        name = (data_cnt['name'][data_cnt['name'] == code].index.tolist())[0]
        return data_cnt[reg].get_value(name)
    else:
        return np.nan

def Sorting(data, reg='region'):
    #if data['Country Name'].get_value(len(data)-5) == np.nan:
    #    data=data.drop(data.index[[range(len(data)-5, len(data))]]).reset_index()
    if 'Region' not in data.columns:
        data['Region'] = data['Country Name'].apply(lambda x: Make_region(x, reg))
        data['Region'].dropna()
        return data.reset_index()
    else:
        return data

In [6]:
data1 = Clearing(data1)
#заметил много шума.
del data1['2016 [YR2016]']

data2 = Clearing(data2)
del data2['2001 [YR2001]']

data3 = Clearing(data3)
del data3['2016 [YR2016]']

Эвристика
Преимущество данного подхода в том, что мы не потеряем единственные значения у некоторых стран. Но при этом, отразим актуальность, а значит и реальность.

In [7]:
def data_inc(data):
    magic = 0.08
    for ind, item in enumerate(np.array(data.columns)):
        if ind not in (range(0,4)):
        #попробую с интерполяцией
            data[item] = data[item].apply(lambda x: ((ind-2)*magic)*x if ((ind-2)*magic)>1 else x).interpolate()
    return data

def R_mean(data):
    if ('Mean' not in data.columns) and ('Region' not in data.columns):
        data['Mean']= data[[item for ind, item in enumerate(np.array(data.columns)) if ind not in (range(0,4))]].apply(lambda x: np.nanmean(x), axis = 1)
    elif ('Region' in data.columns):
        data['Mean']= data[[item for ind, item in enumerate(np.array(data.columns)) if ind not in [(range(0,4)), len(data.columns)-1]]].apply(lambda x: np.nanmean(x), axis = 1)
    return data

Чувствую себя отвратительно за такой код. Доделаю до конца и поправлю. 

In [8]:
#data1 = R_mean(data_inc(data1))
#data2 = R_mean(data_inc(data2))
#data3 = R_mean(data_inc(data3))

Вот тут буду коррелировать, строить графики и пр

In [9]:
def podshape(t1,t2):
    cntrs1 = t1['Country Name'].tolist()
    cntrs2 = t2['Country Name'].tolist()
    x, y = [],[]
    if len(t1)!=len(t2):
        for contr in cntrs1:
            if contr in cntrs2:
                x.append(t1['Mean'][t1['Country Name']== contr].as_matrix()[0])
                y.append(t2['Mean'][t2['Country Name']== contr].as_matrix()[0])
        return x, y
    else:
        return t1['Mean'].as_matrix(), t2['Mean'].as_matrix()

def Shaping(t1,t2): 
    if len(t1)<len(t2):
        return podshape(t1,t2)
    elif len(t1)>len(t2):
        return podshape(t1,t2)
    elif len(t1)==0 or len(t1)==0:
        return True
    else:
        return t1['Mean'].as_matrix(), t2['Mean'].as_matrix()
    
def Correlation(data_r,regions=[], where='region', name='Damn'):
    data = data_r.copy()
    if where == 'region':
        data = Sorting(data)
    else:
        data = Sorting(data, where)
        
    if len(regions)==0:
        regions = data['Region'].dropna().unique()
    
    codes = data['Series Code'].dropna().unique()
    rez=pd.DataFrame()    
    
    tmp = []
    
    for region in regions:
        for code1 in codes:
            for code2 in codes:
                tmp = []
                Sh = Shaping(data[['Mean', 'Country Name']][data['Series Code']==code1][data['Region']==region], data[['Mean', 'Country Name']][data['Series Code']==code2][data['Region']==region])
                if Sh!=True:
                    tmp = sts.pearsonr(Sh[0],Sh[1])
                    
                if len(tmp)!=0 and np.abs(tmp[0])>0.099:
                    rez = rez.append(pd.DataFrame(tmp[0], columns=[code1+':'+region], index=[code2+':'+region+':'+'cor-value']))
                    rez = rez.append(pd.DataFrame(tmp[1], columns=[code1+':'+region], index=[code2+':'+region+':'+'p-value']))
                    rez = rez.groupby(rez.index).first()
        
        filename = name+'.Corr_in_'+str(region)+'.xlsx'
        rez.to_excel(filename, encoding='utf-8')
        rez = pd.DataFrame()
        print (str(region))

In [10]:
#data_cnt['sub-region'].unique()

Correlation(data2, where='region', name='data2')

Correlation(data2, where='sub-region', name='data2')

Correlation(data3, where='region', name='data3')

Correlation(data3, where='sub-region', name='data3')

Correlation(data1, where='region', name='data1')

Correlation(data1, regions =[u'Western Europe', u'Eastern Europe',
       u'Central America', u'Western Africa', u'Northern America',
       u'Southern Africa', u'South-Eastern Asia', u'Eastern Africa',
       u'Eastern Asia', u'Melanesia', u'Micronesia', u'Central Asia'], where='sub-region', name='data1')

Общая таблица

In [11]:
all_data = pd.DataFrame().append(data1).append(data2).append(data2)

In [12]:
years_c = [item for ind, item in enumerate(np.array(data1.columns)) if ind not in (range(0,4)) and item !='Mean']

Correlation(all_data, where='region', name='all')

Correlation(all_data, regions=[u'Western Africa', u'Northern America',
       u'Southern Africa', u'South-Eastern Asia', u'Eastern Africa',
       u'Eastern Asia', u'Melanesia', u'Micronesia', u'Central Asia'], where='sub-region', name='all')

Какое же это фиаско

Продолжаем кодить, на этот раз смотрим по стране без среднего.

In [ ]:
def C_corr(data_r, country='RUS',procent=100):
    #распиши пожалуйста.(Денис)
    data = data_r.copy()
    rez=pd.DataFrame() 
    
    years_c = [item for ind, item in enumerate(np.array(data1.columns)) if ind not in (range(0,4)) and item !='Mean']
    codes = data['Series Code'].dropna().unique()
    
    procent = procent/100
    
    for code1 in codes:
        for code2 in codes:
            X = data[(data['Country Code']==country) & (data['Series Code']==code1)][years_c].as_matrix()
            Y = data[(data['Country Code']==country) & (data['Series Code']==code2)][years_c].as_matrix()
            if len(X)!=0 and len(Y)!=0:
                X,Y=X[0],Y[0]
                
                for i in range(len(X)):
                    if X[i]!=np.nan or Y[i]!=np.nan:
                        #print('OUCH')
                        break
                    elif X[i]!=np.nan or Y[i]!=np.nan:
                        del X[i]
                        del Y[i]
                        
                if len(X)>=len(years_c)*procent and (X.std()>0.7 and Y.std()>0.7):
                    tmp = sts.pearsonr(X,Y)
                    
                    if len(tmp)!=0 and np.abs(tmp[0])>0.099:
                        rez = rez.append(pd.DataFrame(tmp[0], columns=[code1+':'+country], index=[code2+':'+country+':'+'cor-value']))
                        rez = rez.append(pd.DataFrame(tmp[1], columns=[code1+':'+country], index=[code2+':'+country+':'+'p-value']))
                        rez = rez.groupby(rez.index).first()
                    
    filename = 'deep'+'.Corr_in_'+str(country)+'.xlsx'
    print (country)
    rez.to_excel(filename, encoding='utf-8')

In [ ]:
C_corr(all_data)

In [15]:
#for name in all_data['Country Code'].unique():
 #   C_corr(all_data, name)

In [27]:
all_data[(all_data['Series Name'] == 'Improved sanitation facilities (% of population with access)') & (all_data['Country Code']=='RUS')][years_c].as_matrix()[0]

array([ 72.4,  72.4,  72.4,  72.4,  72.3,  72.3,  72.3,  72.3,  72.3,
        72.3,  72.2,  72.2,  72.2,  72.2])

In [26]:
all_data[(all_data['Series Code'] == 'SI.POV.NAHC') & (all_data['Country Code']=='RUS')][years_c].as_matrix()[0]

array([ 24.6,  20.3,  17.6,  17.8,  15.2,  13.3,  13.4,  13. ,  12.5,
        12.7,  10.7,  10.8,  11.2,  13.4])

In [28]:
tmp1, tmp2 = all_data[(all_data['Series Name'] == 'Improved sanitation facilities (% of population with access)') & (all_data['Country Code']=='RUS')][years_c].as_matrix()[0], all_data[(all_data['Series Code'] == 'SI.POV.NAHC') & (all_data['Country Code']=='RUS')][years_c].as_matrix()[0]

(0.83812453430486566, 0.00018153521243417618)

In [32]:
all_data[(all_data['Series Name'] == 'Poverty headcount ratio at national poverty lines (% of population)') & (all_data['Country Code']=='RUS')][years_c]

,2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015]
